# Requirements

In [ ]:
%pip install tensorflow numpy opencv-python firebase_admin scipy

# Imports

In [17]:
import firebase_admin
from firebase_admin import firestore
from firebase_admin import credentials
from google.cloud.firestore_v1.base_query import FieldFilter

import numpy as np
from itertools import combinations
from scipy.spatial import ConvexHull

# Connect with Firestore

In [4]:
CRED = "path/to/key"

In [ ]:
cred = credentials.Certificate(CRED)
firebase_admin.initialize_app(cred)

In [6]:
db = firestore.client()

# Utils

In [43]:
def calculate_convex_hull_area(coords):
    hull = ConvexHull(coords)
    return hull.area

In [44]:
def get_region_area(coordinates:list):
    if len(coordinates) <= 2:
      return 0
    max_area = 0
    # Iterate through all combinations of coordinates
    for i in range(len(coordinates), -1,-1):
        for comb in combinations(coordinates, i):        # Assuming you want to form a triangle (change to 4 for quadrilateral, etc.)
            if not all(c in comb for c in coordinates):  # Skip combinations where not all coordinates are included
                continue
            area = calculate_convex_hull_area(np.array(comb))
            if area > max_area:
                max_area = area
    return max_area

# Fetching Region Maps

In [39]:
region_maps_ = []
region_maps = db.collection('RegionMap').get()

In [40]:
for rm in region_maps:
    conv = rm.to_dict()
    conv["coords"] = [(coord.latitude, coord.longitude) for coord in conv["coords"]]
    conv["central_coord"] = (conv["central_coord"].latitude, conv["central_coord"].longitude)
    region_maps_.append(conv)

# Calculating Convex Hull Area

In [45]:
cc_max_area_pair = []
for i in region_maps_:
    cc_max_area_pair.append((i["central_coord"], get_region_area(i["coords"])))